In [1]:
import os, sys
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:   
    sys.exit("please declare environment variable 'SUMO_HOME'")
    
import sumolib

In [2]:
import pickle

city = "sfc"

with open('data/' + city + '_features.pkl', 'rb') as f:
    features = pickle.load(f)

with open('data/' + city + '_speeds.pkl', 'rb') as f:
    speeds = pickle.load(f)
    
with open('data/small_coords.pkl', 'rb') as f:
    coords = pickle.load(f)

In [3]:
import pandas as pd

edges = pd.read_csv('data/' + city + '.csv', names=["source", "dest"])
edges.head()

,source,dest
0,663028776,32915539
1,349678308,349678300
2,120816570,3971849316
3,31354482,31416669
4,62292066,62292068


In [4]:
# Create node XML
import xml.etree.cElementTree as ET

nodes = ET.Element("nodes")
for n in set(edges["source"]).union(set(edges["dest"])):
    v = str(n)
    long, lat = coords[v]
    node = ET.SubElement(nodes, "node", id=v, x=str(long), y=str(lat))
    
tree = ET.ElementTree(nodes)
tree.write("out/" + city + ".nodes.xml")

In [5]:
# Create edge XML

edgeXML = ET.Element("edges")
i = 0
for i, e in enumerate(edges.itertuples()):
    feat = features[tuple([str(v) for v in e[1:3]])]
    numLanes = str(int(feat[2]))
    ET.SubElement(edgeXML, "edge", {"id": str(i), "from": str(e[1]), "to": str(e[2]), 
                                    "numLanes": str(int(feat[2])), "speed": str(feat[3] * 0.44704), "length": str(feat[4])})
    
tree = ET.ElementTree(edgeXML)
tree.write("out/" + city + ".edges.xml")

In [6]:
import xml.etree.cElementTree as ET
import numpy as np

net = sumolib.net.readNet('out/' + city + '.net.xml')

with open('data/' + city + '_flows.pkl', 'rb') as f:
    flows = pickle.load(f)
    
data = ET.Element("data")

for t in range(1):
    interval = ET.SubElement(data, "interval", id=str(t), begin=str(3600 * t), end=str(3600 * (t + 1)))
    
    for i, e in enumerate(net.getEdges()):
        key = (str(e.getFromNode().getID()), str(e.getToNode().getID()))
        if key in flows.keys():
            print(key)
            flow = str(int(flows[key]))
            try:
                speed = speeds[key] * 0.44704 # Convert from mph to m/s
                ET.SubElement(interval, "edge", id=str(e.getID()), entered=flow, speed=str(speed))
            except KeyError:
                ET.SubElement(interval, "edge", id=str(e.getID()), entered=flow)
            

('358392935', '52995189')
('295958025', '53043955')
('3971341237', '366486421')
('110394877', '110394875')
('1400725390', '53138228')
('734793165', '734793166')
('256218041', '256218042')
('4563755435', '4563755436')
('295570840', '52995643')
('57784714', '57956665')
('263024163', '1807996341')
('33154452', '33154454')
('349877339', '349877364')
('365488418', '65367200')
('4347599125', '6814468451')
('1243945808', '720620515')
('312407551', '259001460')
('263015126', '734803479')
('263015150', '651116150')
('3958258216', '53013485')
('32914630', '3960001767')
('318856209', '318856210')
('355703807', '1222817967')
('53064146', '53013480')
('358391108', '601626927')
('2447255569', '3102940199')
('29892678', '272140172')
('1299142111', '1299142105')
('720619039', '3974638968')
('310996456', '1760094752')
('35833145', '35833146')
('57948867', '1215707915')
('57830853', '353477608')
('1316775942', '1316775952')
('62207026', '4382564515')
('257810782', '2391184681')
('312388006', '312387699'

('1316891227', '1316891051')
('57858194', '1222967836')
('369930180', '3982938787')
('1684055515', '257389852')
('262911223', '1067563086')
('2287663083', '65485287')
('887717954', '887717956')
('1301679854', '256898640')
('4381380520', '62487882')
('1722591695', '62261508')
('110461801', '110461809')
('65281752', '65281888')
('65402644', '3706195797')
('663024722', '32915482')
('1672922582', '1672922606')
('57949044', '57793502')
('4393710977', '62486950')
('346682284', '6193840204')
('1087346730', '1087346731')
('355814957', '56028156')
('257389894', '721451744')
('62486267', '62486269')
('310986744', '310986891')
('32791310', '1316890786')
('4388594535', '4388594532')
('1292989497', '65355466')
('311915293', '1299283771')
('62487760', '62487768')
('242862789', '242862790')
('263645141', '110461697')
('657607461', '263015642')
('258120067', '740415463')
('242864685', '242864686')
('1389052654', '258179932')
('65337860', '2315556017')
('4381380332', '62487789')
('240487106', '24048710

In [ ]:
tree = ET.ElementTree(data)
tree.write("out/routeSampler/" + city + ".edgedata.xml")